In [16]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urljoin
import re
from datetime import datetime
import mysql.connector
from mysql.connector import errorcode
import time


In [17]:
conn = mysql.connector.connect(
   user= "wsa",
   host= "34.68.250.121",
   database = "ufcElo",
   password = "LeBron>MJ!"
)

cursor = conn.cursor()

In [18]:
def insert_event(name, date):
    cursor.execute("INSERT INTO events (name, date) VALUES (%s, %s)", (name, date))
    conn.commit()
    return cursor.lastrowid

In [19]:
def insert_fight(event_id, winner_id, loser_id, weight_class, title_fight, method, round_end, fight_time):
    #print(f"event_id: {event_id}, winner_id: {winner_id}, loser_id: {loser_id}")
    #print(f"weight_class: {weight_class}, title_fight: {title_fight}, method: {method}")
    #print(f"round_end: {round_end}, fight_time: {fight_time}")

    # Ensure method is a string (in case it's still a list)
    cursor.execute("""
        INSERT INTO fights (event_id, fighter1_id, fighter2_id, weight_class, result, method, rounds, time)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
    """, (event_id, winner_id, loser_id, weight_class, title_fight, method, round_end, fight_time))
    conn.commit()

In [20]:
def insert_fighter(fighter_name, weight_class):
    # Check if the fighter already exists in the database
    cursor.execute("SELECT fighter_id FROM fighters WHERE name = %s", (fighter_name,))
    result = cursor.fetchone()

    if result:
        # Fighter already exists, return the fighter_id
        return result[0]
    else:
        # If the fighter does not exist, insert them into the table
        cursor.execute("""
            INSERT INTO fighters (name, weightclass, wins, losses, draws, currentElo)
            VALUES (%s, %s, %s, %s, %s, %s)
        """, (fighter_name, weight_class, 0, 0, 0, 1500))  # Assuming default Elo is 1500
        conn.commit()

        # Return the fighter_id of the newly inserted fighter
        return cursor.lastrowid

In [21]:
url = requests.get('http://www.ufcstats.com/statistics/events/completed?page=all')
soup = BeautifulSoup(url.text, 'html.parser')
eventLinks = []
for link in soup.select(".b-statistics__table-row a"):
    eventUrl = link["href"]
    eventLinks.append(eventUrl)

#for item in eventLinks:
    #print(item)

In [22]:
def update_fighter_record(fighter_name, result):
    retries = 3  # Retry a few times if a deadlock occurs
    attempt = 0
    while attempt < retries:
        try:
            if result == "win":
                cursor.execute("""
                    UPDATE fighters
                    SET wins = wins + 1, currentElo = currentElo + 10  # Elo increase for a win
                    WHERE name = %s
                """, (fighter_name,))
            elif result == "loss":
                cursor.execute("""
                    UPDATE fighters
                    SET losses = losses + 1, currentElo = currentElo - 10  # Elo decrease for a loss
                    WHERE name = %s
                """, (fighter_name,))
            elif result == "draw":
                cursor.execute("""
                    UPDATE fighters
                    SET draws = draws + 1
                    WHERE name = %s
                """, (fighter_name,))
            elif result == "no contest":
                cursor.execute("""
                    UPDATE fighters
                    SET draws = draws + 1
                    WHERE name = %s
                """, (fighter_name,))

            conn.commit()  # Commit the changes if successful
            break  # Exit the loop if successful

        except mysql.connector.Error as err:
            if err.errno == errorcode.ER_LOCK_DEADLOCK:
                print(f"Deadlock detected. Retrying... (Attempt {attempt + 1}/{retries})")
                attempt += 1
                time.sleep(1)  # Wait before retrying
            else:
                print(f"Error: {err}")
                conn.rollback()  # Rollback the transaction in case of a different error
                break  # Exit the loop if it's not a deadlock

In [23]:
def get_fighter_id(fighter_name):
    # Query to get fighter_id by name (adjust based on your database schema)
    cursor.execute("SELECT fighter_id FROM fighters WHERE fighter_name = ?", (fighter_name,))
    
    # Fetch the result
    result = cursor.fetchone()

    # If the fighter exists in the database, return their fighter_id
    if result:
        return result[0]  # Assuming fighter_id is the first column in the result
    else:
        # If the fighter is not found, return None or handle it differently
        print(f"Fighter '{fighter_name}' not found in the database.")
        return None

In [24]:
def insert_advanced_stats(fight_id, fighter_id, advanced_stats):
    # Inserting into the advanced stats table
    cursor.execute("""
        INSERT INTO fight_advanced_stats (fight_id, fighter_id, head_strikes, body_strikes, leg_strikes, distance_strikes, clinch_strikes, ground_strikes)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
    """, (
        fight_id, fighter_id,
        advanced_stats['head'],
        advanced_stats['body'],
        advanced_stats['leg'],
        advanced_stats['distance'],
        advanced_stats['clinch'],
        advanced_stats['ground']
    ))

    # Commit and close connection
    conn.commit()
    conn.close()

In [25]:
def get_fighter_stats(fight_url, fighter1_id, fighter2_id):
    """
    Scrape the advanced stats for both fighter1 and fighter2.
    """
    url = fight_url
    headers = {"User-Agent": "Mozilla/5.0"}

    # Fetch the webpage
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")

    # Find all tables
    tables = soup.find_all("table")

    fight_stats_table = tables[0]  # Replace with the correct table index
    fight_stats_table2 = tables[2]  # Replace with the correct table index
    rows = fight_stats_table.find_all("tr")
    rows2 = fight_stats_table2.find_all("tr")
    
    fighter_stats = {
        'fighter1': {'fighter_id': fighter1_id, 'stats': {}},
        'fighter2': {'fighter_id': fighter2_id, 'stats': {}}
    }

    # Process the stats from fight_stats_table
    for row in rows:
        cols = row.find_all("td")
        if len(cols) > 1:
            # Process the fighter names and stats
            fighter_names = [name.strip() for name in cols[0].text.split("\n") if name.strip()]
            if len(fighter_names) == 2:
                fighter1_name = fighter_names[0].strip()
                fighter2_name = fighter_names[1].strip()

                # Stats for fighter1
                stats = [col.text.strip().replace("\n", " ") for col in cols[1:]]
                fighter_stats['fighter1']['stats'] = {
                    'kd': stats[0].split()[0],  # Knockdowns
                    'sig_str': stats[1].split()[0],  # Significant Strikes
                    'sig_str_pct': stats[2].split()[0],  # Significant Strikes %
                    'total_str': stats[3].split()[0],  # Total Strikes
                    'td': stats[4].split()[0],  # Takedowns
                    'td_pct': stats[5].split()[0],  # Takedown %
                    'sub_att': stats[6].split()[0],  # Submission Attempts
                    'rev': stats[7].split()[0],  # Reversals
                    'ctrl': stats[8].split()[0],  # Control Time
                }

                # Stats for fighter2 (same as fighter1, but second stat)
                fighter_stats['fighter2']['stats'] = {
                    'kd': stats[0].split()[1],
                    'sig_str': stats[1].split()[3],
                    'sig_str_pct': stats[2].split()[1],
                    'total_str': stats[3].split()[3],
                    'td': stats[4].split()[3],
                    'td_pct': stats[5].split()[1],
                    'sub_att': stats[6].split()[1],
                    'rev': stats[7].split()[1],
                    'ctrl': stats[8].split()[1],
                }

    # Process advanced stats (from fight_stats_table2)
    for row in rows2:
        cols = row.find_all("td")
        if len(cols) > 1:
            # Process the advanced stats for each fighter
            stats = [col.text.strip().replace("\n", " ") for col in cols[1:]]
            fighter_stats['fighter1']['stats'].update({
                'head': stats[2].split()[0],
                'body': stats[3].split()[0],
                'leg': stats[4].split()[0],
                'distance': stats[5].split()[0],
                'clinch': stats[6].split()[0],
                'ground': stats[7].split()[0]
            })

            fighter_stats['fighter2']['stats'].update({
                'head': stats[2].split()[3],
                'body': stats[3].split()[3],
                'leg': stats[4].split()[3],
                'distance': stats[5].split()[3],
                'clinch': stats[6].split()[3],
                'ground': stats[7].split()[3]
            })

    # Now store the stats for both fighters
    insert_advanced_stats(fight_id, fighter_stats['fighter1']['fighter_id'], fighter_stats['fighter1']['stats'])
    insert_advanced_stats(fight_id, fighter_stats['fighter2']['fighter_id'], fighter_stats['fighter2']['stats'])

    return fighter_stats

In [26]:
fightData = []

for eventUrl in eventLinks[0:600]:
    eventUrlEvent = requests.get(eventUrl)
    eventSoup = BeautifulSoup(eventUrlEvent.text, "html.parser")

    # get event name & date
    eventName = eventSoup.select_one(".b-content__title").text.strip()
    eventDateRaw = eventSoup.select_one(".b-list__box-list-item:first-child").text.strip()
    eventDateClean = re.search(r"([A-Za-z]+ \d{1,2}, \d{4})", eventDateRaw)
    
    if eventDateClean:
        eventDate = datetime.strptime(eventDateClean.group(1), "%B %d, %Y").strftime("%Y-%m-%d")
    else:
        eventDate = None  # Handle cases where date isn't found properly

    #event ID
    event_id = insert_event(eventName, eventDate)
    
    # find the fight table
    fightTable = eventSoup.find("table", class_="b-fight-details__table")
    
    if fightTable:
        # loop through all the rows and extract the relevant data
        for row in fightTable.find_all("tr")[1:]:  # Skip header row
            cols = [col.text.strip() for col in row.find_all("td")]
            if len(cols) > 6:
                winnerTagFake = cols[0].split("\n")
                winnerTag = winnerTagFake[0].strip()
                #print(f"DEBUG - winnerTag: {winnerTag}")  # Debugging output to check winnerTag

                fighters = [fighter.strip() for fighter in cols[1].split("\n") if fighter.strip()]
                
                winner = fighters[0] if fighters else "Unknown" 
                loser = fighters[-1] if len(fighters) > 1 else "Unknown"  
                weightClass = cols[6]
                method = [m.strip() for m in cols[7].split("\n") if m.strip()]  # Strip and remove empty strings
                trueMethod = method[0] if method else "Unknown"  # Get the first valid method, or "Unknown"
                roundEnd = cols[8] if len(cols) > 8 else "Unknown"
                rounds = int(roundEnd.split(" ")[0])
                time = cols[9] if len(cols) > 9 else "Unknown"
                titleFight = "No"
                beltImage = row.find("img", src=lambda x: x and "belt.png" in x)
                
                if beltImage:
                    titleFight = "Yes"  # If the belt image is found, it's a title fight

                # Insert or get fighter IDs
                winner_id = insert_fighter(winner, weightClass)
                loser_id = insert_fighter(loser, weightClass)

                # Determine the fight result (win, loss, draw, no contest)
                result = "win"  # Default to win for the winner
                if "draw" in cols[0]:
                    result = "draw"
                elif "nc" in cols[0]:
                    result = "no contest"

                fighter1_id = get_fighter_id(winner)  # Replace with your actual logic to get fighter1_id
                fighter2_id = get_fighter_id(loser)
                
                # Update the fighter records based on the result
                if result == "win":
                    update_fighter_record(winner, "win")
                    update_fighter_record(loser, "loss")
                elif result == "draw":
                    update_fighter_record(winner, "draw")
                    update_fighter_record(loser, "draw")
                elif result == "nc":
                    update_fighter_record(winner, "no contest")
                    update_fighter_record(loser, "no contest")

                # Insert fight into MySQL
                insert_fight(event_id, winner_id, loser_id, weightClass, result, trueMethod, roundEnd, time)
                fight_url = f"http://www.ufcstats.com/fight-details/{cols[0]}"  # Assuming fight ID is in the first column
                get_fighter_stats(fight_url, winner_id, loser_id)

               
                # Append fight data to the list
                fightData.append([winnerTag, winner, loser, weightClass, titleFight, trueMethod, roundEnd, time, result])


ProgrammingError: Not all parameters were used in the SQL statement

In [ ]:
initialElo = 1500
kFactor = 50 
eloRatings = {}
maxEloRating = {}

In [ ]:
def expectedScore(ratingA, ratingB):
    return 1 / (1 + 10 ** ((ratingB - ratingA) / 400))

In [ ]:
def updateElo(winner, loser, eloRatings, kFactor, isTitleFight):
    if winner not in eloRatings:
        eloRatings[winner] = initialElo
        maxEloRating[winner] = initialElo
    if loser not in eloRatings:
        eloRatings[loser] = initialElo
        maxEloRating[loser] = initialElo

    ratingWinner = eloRatings[winner]
    ratingLoser = eloRatings[loser]

    # calculate expected scores
    expectedWinner = expectedScore(ratingWinner, ratingLoser)
    expectedLoser = expectedScore(ratingLoser, ratingWinner)

    # Update Elo ratings
    if isTitleFight == "Yes":   
        newWinnerElo = ratingWinner + (kFactor * (1 - expectedWinner) * 2)
        newLoserElo = ratingLoser + kFactor * (0 - expectedLoser)
        if newWinnerElo > maxEloRating[winner]:
            eloRatings[winner] = newWinnerElo
            maxEloRating[winner] = newWinnerElo
        else:
            eloRatings[loser] = newLoserElo
            eloRatings[winner] = newWinnerElo
    else:
        newWinnerElo = ratingWinner + kFactor * (1 - expectedWinner)
        newLoserElo = ratingLoser + kFactor * (0 - expectedLoser)
        if newWinnerElo > maxEloRating[winner]:
            eloRatings[winner] = newWinnerElo
            maxEloRating[winner] = newWinnerElo
        else:
            eloRatings[loser] = newLoserElo
            eloRatings[winner] = newWinnerElo

In [ ]:
def processFights(fight_data):
    for fight in reversed(fightData):
        winnerTagFake = fight[0].split("\n")
        winnerTag = winnerTagFake[0].strip()
        winner = fight[1]
        loser = fight[2]
        isTitleFight = fight[4]
        if winnerTag != "nc" and winnerTag != "draw" :
            updateElo(winner, loser, eloRatings, kFactor, isTitleFight)

In [ ]:
processFights(fightData)
sortedElo = sorted(maxEloRating.items(), key=lambda x: x[1], reverse=True)
for fighter, rating in sortedElo:
    print(f"{fighter}: {rating:.2f}")

Jon Jones: 2107.16
Georges St-Pierre: 2065.20
Islam Makhachev: 2056.78
Max Holloway: 2046.13
Kamaru Usman: 2015.77
Alexander Volkanovski: 2015.69
Stipe Miocic: 2001.93
Charles Oliveira: 1996.93
Leon Edwards: 1992.20
Israel Adesanya: 1983.59
Rafael Dos Anjos: 1979.13
Aljamain Sterling: 1969.98
Anderson Silva: 1969.93
Jose Aldo: 1968.93
Dustin Poirier: 1963.59
Frankie Edgar: 1960.32
Amanda Nunes: 1955.32
Jim Miller: 1949.64
Daniel Cormier: 1942.99
Donald Cerrone: 1937.83
Chris Weidman: 1935.57
Valentina Shevchenko: 1933.79
Michael Bisping: 1929.24
Khabib Nurmagomedov: 1928.58
Demetrious Johnson: 1923.40
Belal Muhammad: 1918.04
Robbie Lawler: 1917.31
Dricus Du Plessis: 1913.83
Neil Magny: 1912.76
Andrei Arlovski: 1911.92
Demian Maia: 1907.34
Zhang Weili: 1906.07
Robert Whittaker: 1904.59
Sean Strickland: 1897.77
Tony Ferguson: 1895.27
Glover Teixeira: 1895.18
Deiveson Figueiredo: 1894.28
Francis Ngannou: 1892.72
Chuck Liddell: 1892.22
BJ Penn: 1891.88
Randy Couture: 1888.61
TJ Dillashaw: 

In [ ]:
for item in fightData:
    print(item)

['win', 'Sean Brady', 'Leon Edwards', 'Welterweight', 'No', 'SUB', '4', '1:39']
['win', 'Carlos Ulberg', 'Jan Blachowicz', 'Light Heavyweight', 'No', 'U-DEC', '3', '5:00']
['win', 'Kevin Holland', 'Gunnar Nelson', 'Welterweight', 'No', 'U-DEC', '3', '5:00']
['win', 'Alexia Thainara', 'Molly McCann', "Women's Strawweight", 'No', 'SUB', '1', '4:32']
['win', 'Chris Duncan', 'Jordan Vucenic', 'Lightweight', 'No', 'SUB', '2', '3:42']
['win', 'Nathaniel Wood', 'Morgan Charriere', 'Featherweight', 'No', 'U-DEC', '3', '5:00']
['win', 'Chris Padilla', 'Jai Herbert', 'Lightweight', 'No', 'S-DEC', '3', '5:00']
['win', "Lone'er Kavanagh", 'Felipe dos Santos', 'Flyweight', 'No', 'U-DEC', '3', '5:00']
['win', 'Marcin Tybura', 'Mick Parkin', 'Heavyweight', 'No', 'U-DEC', '3', '5:00']
['win', 'Christian Leroy Duncan', 'Andrey Pulyaev', 'Middleweight', 'No', 'U-DEC', '3', '5:00']
['win', 'Shauna Bannon', 'Puja Tomar', "Women's Strawweight", 'No', 'SUB', '2', '3:22']
['win', 'Caolan Loughran', 'Nathan F